Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Introduction to Azure Machine Learning: Deploy web service

In previous example, you ran an experiment to estimate value of pi. In this example, we'll go a step further and use your estimated value to create a web service that computes the area of a circle in a real time.

First, let's load the workspace, and retrieve the latest run from your experiment.

In [ ]:
from azureml.core import Workspace, Experiment, Run
import math, random, pickle, json

In [ ]:
ws = Workspace.from_config()

In [ ]:
experiment_name = "my-first-experiment"
run = list(Experiment(workspace = ws, name = experiment_name).get_runs())[0]

In the previous example you saved a file containing the pi value into run history. Let's register that file as a model: a versioned asset in your workspace that is used to build web services.

In [ ]:
model = run.register_model(model_name = "pi_estimate", model_path = "outputs/pi_estimate.txt")

In order to deploy a web service, you'll also need to create a scoring script that consists of two parts: an *init* method that loads the model, and a *run* method that gets invoked when you call the web service.

In [ ]:
%%writefile score.py
import pickle, json
from azureml.core.model import Model

def init():
    global pi_estimate
    model_path = Model.get_model_path(model_name = "pi_estimate")
    with open(model_path, "rb") as f:
        pi_estimate = float(pickle.load(f))

def run(raw_data):
    try:
        radius = json.loads(raw_data)["radius"]
        result = pi_estimate * radius**2
        return json.dumps({"area": result})
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

You also need to specify the library dependencies of your scoring script as conda yml file. Let's simply use Azure ML's default dependencies.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

CondaDependencies().save_to_file(".", "myenv.yml")

Then, let's deploy the web service on Azure Container Instance: a serverless compute for running Docker images. Azure ML service takes care of packaging your model, scoring script and dependencies into Docker image and deploying it.

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.image import ContainerImage

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)


image_config = ContainerImage.image_configuration(execution_script = "score.py", 
                                    runtime = "python", 
                                    conda_file = "myenv.yml")

service = Webservice.deploy_from_model(name = "area-calculator",
                                       deployment_config = aci_config,
                                       models = [model],
                                       image_config = image_config,
                                       workspace = ws)
service.wait_for_deployment(show_output = True)

You can try out the web service by passing in data as json-formatted request. Run the cell below and move the slider around to see real-time responses.

In [ ]:
from ipywidgets import interact

def get_area(radius):
    request = json.dumps({"radius": radius})
    response = service.run(input_data = request)
    return json.loads(response)["area"]

interact(get_area,radius=(0,10))

Finally, delete the web service once you're done

In [ ]:
service.delete()

As your next step, take a look at the more detailed tutorial for building an image classification model using Azure Machine Learning service.